In [165]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier, Perceptron, PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import pickle

In [166]:
df = pd.read_csv('ads dataset.csv')
df = df.iloc[0:600] # get only clicked ads
df

,Age,Male,Income,Area,Extrovert,Weekday,Time,Biller_Meralco,Biller_PLDT,Biller_Maynilad,Total Outstanding Loan,Linked Bank Account,Ave Transaction Count,SD Transaction Count,Push notif,Banner,Popup,Clicked
0,18,1,13279,Rural,1,1,Evening,0,1,0,6256,0,20,2,Sports,Sports,Sports,1
1,20,1,14428,Rural,1,1,Evening,0,1,1,7635,0,27,3,Sports,Sports,Sports,1
2,21,1,16785,Rural,1,1,Evening,1,0,1,8500,0,86,3,Sports,Sports,Sports,1
3,19,1,14912,Rural,1,1,Evening,1,0,0,5632,1,43,5,Sports,Sports,Sports,1
4,18,1,13183,Rural,1,1,Evening,1,1,0,5360,0,24,10,Sports,Sports,Sports,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,36,1,59296,Suburban,0,0,Midnight,1,0,0,5018,1,66,9,Fashion,Travel,Sports,1
596,49,1,45055,Rural,0,1,Midnight,1,1,1,9318,1,76,7,Travel,Fashion,Fashion,1
597,44,1,57379,Suburban,0,0,Midnight,1,1,1,5709,1,2,1,Fashion,Finance,Finance,1
598,42,0,53132,Rural,0,1,Midnight,1,1,0,8415,0,38,1,Fashion,Finance,Sports,1


In [167]:
df = df.drop('Clicked', axis=1)
X = df.drop(['Push notif',	'Banner',	'Popup'], axis=1)

In [168]:
X

,Age,Male,Income,Area,Extrovert,Weekday,Time,Biller_Meralco,Biller_PLDT,Biller_Maynilad,Total Outstanding Loan,Linked Bank Account,Ave Transaction Count,SD Transaction Count
0,18,1,13279,Rural,1,1,Evening,0,1,0,6256,0,20,2
1,20,1,14428,Rural,1,1,Evening,0,1,1,7635,0,27,3
2,21,1,16785,Rural,1,1,Evening,1,0,1,8500,0,86,3
3,19,1,14912,Rural,1,1,Evening,1,0,0,5632,1,43,5
4,18,1,13183,Rural,1,1,Evening,1,1,0,5360,0,24,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,36,1,59296,Suburban,0,0,Midnight,1,0,0,5018,1,66,9
596,49,1,45055,Rural,0,1,Midnight,1,1,1,9318,1,76,7
597,44,1,57379,Suburban,0,0,Midnight,1,1,1,5709,1,2,1
598,42,0,53132,Rural,0,1,Midnight,1,1,0,8415,0,38,1


In [169]:
y = df[['Push notif',	'Banner',	'Popup']]
X = pd.get_dummies(X, dtype='int')

In [171]:
X.columns

Index(['Age', 'Male', 'Income', 'Extrovert', 'Weekday', 'Biller_Meralco',
       'Biller_PLDT', 'Biller_Maynilad', 'Total Outstanding Loan',
       'Linked Bank Account', 'Ave Transaction Count', 'SD Transaction Count',
       'Area_Rural', 'Area_Suburban', 'Area_Urban', 'Time_Afternoon',
       'Time_Evening', 'Time_Midnight', 'Time_Morning'],
      dtype='object')

In [172]:
y

,Push notif,Banner,Popup
0,Sports,Sports,Sports
1,Sports,Sports,Sports
2,Sports,Sports,Sports
3,Sports,Sports,Sports
4,Sports,Sports,Sports
...,...,...,...
595,Fashion,Travel,Sports
596,Travel,Fashion,Fashion
597,Fashion,Finance,Finance
598,Fashion,Finance,Sports


In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((420, 19), (420, 3), (180, 19), (180, 3))

In [174]:
X_day = []
y_day = []
mult = 42 # data per day
for i in range(10): # 10 days
    X_day.append(X_train.iloc[i*mult:(i+1)*mult])
    y_day.append(y_train.iloc[i*mult:(i+1)*mult])

mnb = MultinomialNB()
model = MultiOutputClassifier(mnb)
model.fit(X_day[0], y_day[0]) # day 0

ctr_push = []
ctr_banner = []
ctr_popup = []
df_pred = pd.DataFrame(model.predict(X_day[1]), columns=['Push notif_pred', 'Banner_pred', 'Popup_pred'])
df_pred[['Push notif', 'Banner', 'Popup']] = y_day[1].to_numpy()
df_pred['Push notif_equal'] = (df_pred['Push notif_pred'] == df_pred['Push notif'])
df_pred['Banner_equal'] = (df_pred['Banner_pred'] == df_pred['Banner'])
df_pred['Popup_equal'] = (df_pred['Popup_pred'] == df_pred['Popup'])
df_pred[df_pred.columns[-3:]] = df_pred[df_pred.columns[-3:]].astype(int)
print('Day 0 Push notif CTR: ', df_pred['Push notif_equal'].sum()/len(df_pred.index))
print('Day 0 Banner CTR: ', df_pred['Banner_equal'].sum()/len(df_pred.index))
print('Day 0 Popup CTR: ', df_pred['Popup_equal'].sum()/len(df_pred.index))
ctr_push.append(df_pred['Push notif_equal'].sum()/len(df_pred.index))
ctr_banner.append(df_pred['Banner_equal'].sum()/len(df_pred.index))
ctr_popup.append(df_pred['Popup_equal'].sum()/len(df_pred.index))

Day 0 Push notif CTR:  0.35714285714285715
Day 0 Banner CTR:  0.6190476190476191
Day 0 Popup CTR:  0.35714285714285715


In [175]:
# implement incremental/online learning as days go by
for i in range(1,9):
    model.partial_fit(X_day[i], y_day[i]) # day 1 onwards

    df_pred = pd.DataFrame(model.predict(X_day[i]), columns=['Push notif_pred', 'Banner_pred', 'Popup_pred'])
    df_pred[['Push notif', 'Banner', 'Popup']] = y_day[i].to_numpy()
    df_pred['Push notif_equal'] = (df_pred['Push notif_pred'] == df_pred['Push notif'])
    df_pred['Banner_equal'] = (df_pred['Banner_pred'] == df_pred['Banner'])
    df_pred['Popup_equal'] = (df_pred['Popup_pred'] == df_pred['Popup'])
    df_pred[df_pred.columns[-3:]] = df_pred[df_pred.columns[-3:]].astype(int)
    print('Day '+str(i)+' Push notif CTR: ', df_pred['Push notif_equal'].sum()/len(df_pred.index))
    print('Day '+str(i)+' Banner CTR: ', df_pred['Banner_equal'].sum()/len(df_pred.index))
    print('Day '+str(i)+' Popup CTR: ', df_pred['Popup_equal'].sum()/len(df_pred.index))
    ctr_push.append(df_pred['Push notif_equal'].sum()/len(df_pred.index))
    ctr_banner.append(df_pred['Banner_equal'].sum()/len(df_pred.index))
    ctr_popup.append(df_pred['Popup_equal'].sum()/len(df_pred.index))

Day 1 Push notif CTR:  0.5714285714285714
Day 1 Banner CTR:  0.6190476190476191
Day 1 Popup CTR:  0.38095238095238093
Day 2 Push notif CTR:  0.6904761904761905
Day 2 Banner CTR:  0.6428571428571429
Day 2 Popup CTR:  0.5714285714285714
Day 3 Push notif CTR:  0.5714285714285714
Day 3 Banner CTR:  0.6190476190476191
Day 3 Popup CTR:  0.5
Day 4 Push notif CTR:  0.5952380952380952
Day 4 Banner CTR:  0.5952380952380952
Day 4 Popup CTR:  0.6190476190476191
Day 5 Push notif CTR:  0.5714285714285714
Day 5 Banner CTR:  0.5238095238095238
Day 5 Popup CTR:  0.5238095238095238
Day 6 Push notif CTR:  0.5
Day 6 Banner CTR:  0.5238095238095238
Day 6 Popup CTR:  0.5952380952380952
Day 7 Push notif CTR:  0.5714285714285714
Day 7 Banner CTR:  0.5714285714285714
Day 7 Popup CTR:  0.6190476190476191
Day 8 Push notif CTR:  0.38095238095238093
Day 8 Banner CTR:  0.38095238095238093
Day 8 Popup CTR:  0.5238095238095238


In [176]:
pickle.dump(model, open('recommender.pkl', 'wb'))